In [6]:
# Before delving into the implementation of YOLOv8 for our Proof of Concept (PoC), 
# it's crucial to consider a few key aspects regarding the dataset provided by Spare-it. 
# There are notable characteristics we must address to ensure the success of our model.

# Firstly, our dataset exhibits class imbalance, where certain recyclable 
# classes are underrepresented or absent entirely. For instance, labels 
# like "paper towel" may occur frequently, while others may be scarce. 
# To mitigate this imbalance, we plan to leverage pretrained models. 
# These models not only help address class imbalances but also enhance the 
# overall accuracy of our model as Spare-it's dataset expands and evolves.

# Furthermore, within our dataset, we encounter a distinction between material 
# classes and specific names (e.g., "paper towel"). Notably, the material class 
# serves as a superclass encompassing various specific names. Therefore, achieving 
# optimal performance on these specific names is crucial for enhancing the overall model accuracy.

# Our initial step involves transforming Spare-it's dataset, currently in cocojson 
# format, to align with the requirements of YOLOv8's dataset. Initially, we prioritize 
# handling bounding boxes (bbox) and defer addressing segmentation aspects until we achieve satisfactory accuracy levels.

In [1]:
# You will need to have images folder with all the images and cocojson folder with all the labels to run codes below.
import json
import os

# Your directories
coco_dir = './cocojson'  # Directory containing COCO JSON files
labels_dir = './labels'  # Final directory to save the YOLO format files

# These are index mapping we have to do.
# YOLOv8 needs your name starting from 0 incrementing by 1.
# Spare-it has id values not in this format, so we have mapped them in this way.
id_to_index = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7, 10: 8, 11: 9, 
               14: 10, 15: 11, 16: 12, 17: 13, 18: 14, 19: 15, 115: 16, 114: 17, 
               116: 18, 117: 19, 122: 20, 123: 21, 124: 22, 127: 23, 20: 24, 21: 25, 
               22: 26, 23: 27, 24: 28, 25: 29, 26: 30, 27: 31, 30: 32, 125: 33, 126: 34, 
               36: 35, 38: 36, 40: 37, 43: 38, 44: 39, 45: 40, 46: 41, 47: 42, 48: 43, 
               49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 
               58: 53, 59: 54, 60: 55, 120: 56, 121: 57, 63: 58, 65: 59, 66: 60, 67: 61, 
               68: 62, 69: 63, 70: 64, 72: 65, 73: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 
               82: 72, 83: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 118: 79, 119: 80, 89: 81, 
               91: 82, 92: 83, 93: 84, 94: 85, 95: 86, 96: 87, 97: 88, 98: 89, 99: 90, 100: 91, 
               101: 92, 102: 93, 103: 94, 106: 95, 107: 96, 109: 97, 110: 98, 112: 99}

# Function to read Spare-it's custom COCO json into standardized YOLO format.
# I am basing this off from Ultralytics' JSON2YOLO.
# Reason building custom one is to have the id_to_index mapping within coco_to_yolo.
def coco_to_yolo(coco_json_path, labels_dir, id_to_index):
    with open(coco_json_path) as f:
        data = json.load(f)

    # We will need image_width, image_height, and overall annotation here
    for image in data['images']:
        image_width, image_height = image['width'], image['height']
        yolo_annotations = [] # our converted yolo data format

        for annotation in data['annotations']:
            category_id = id_to_index.get(annotation['category_id'], -1) # convert category_id using the id_to_index mapping
            x_min, y_min, bbox_width, bbox_height = annotation['bbox']
            x_center = x_min + (bbox_width / 2)
            y_center = y_min + (bbox_height / 2)

            x_center_normalized = x_center / image_width
            y_center_normalized = y_center / image_height
            width_normalized = bbox_width / image_width
            height_normalized = bbox_height / image_height

            yolo_format = f"{category_id} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}"
            yolo_annotations.append(yolo_format)

        base_filename = os.path.splitext(os.path.basename(coco_json_path))[0]
        output_filename = os.path.join(labels_dir, f"{base_filename}.txt")
        
        # Write output into YOLO format
        with open(output_filename, 'w') as f:
            for item in yolo_annotations:
                f.write("%s\n" % item)

# Creates labels directory and iterate over all cocojson files to that folder
def convert_directory(coco_dir, labels_dir, id_to_index):
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)

    for filename in os.listdir(coco_dir):
        if filename.endswith(".json"):
            coco_json_path = os.path.join(coco_dir, filename)
            coco_to_yolo(coco_json_path, labels_dir, id_to_index)


convert_directory(coco_dir, labels_dir, id_to_index)

In [2]:
# Now we have all the data formatted to conduct training the model.
#
# We will need to use the dataset.yml, I have generated.
# Looks through the YOLO format id to get the label name and figure out where the train and val images are at.
# We will need to create a main folder called datasets. Inside this there will be train and val folders. Inside each
# of these three folder, we will have images and labels folder. By doing this you will be able to customize YOLOv8 model with own dataset.
import os
import shutil
import random

# Generate YOLO custom dataset format
subfolders = ['train', 'val']
for subfolder in subfolders:
    os.makedirs(os.path.join('./datasets', subfolder, 'images'), exist_ok=True)
    os.makedirs(os.path.join('./datasets', subfolder, 'labels'), exist_ok=True)

# Base dataset to create these splits
labels_path = './labels'
images_path = './images'
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']

label_files = [file for file in os.listdir(labels_path) if file.endswith('.txt')]
random.shuffle(label_files)  # Randomize the list

# Calculate split sizes
# Change the percentange for your need
num_total = len(label_files)
num_train = int(num_total * 0.85)
num_val = num_total - num_train

# Split the files
train_files = label_files[:num_train]
val_files = label_files[num_train:]

def copy_files(files, source_path_images, source_path_labels, target_path_images, target_path_labels):
    for file in files:
        base_filename = os.path.splitext(file)[0]
        # Attempt to find and copy the first matching image file with any of the specified extensions
        copied = False
        for ext in image_extensions:
            image_file = base_filename + ext
            if os.path.exists(os.path.join(source_path_images, image_file)):
                shutil.copy(os.path.join(source_path_images, image_file),
                            os.path.join(target_path_images, image_file))
                copied = True
                break  # stops after first match
        if not copied:
            print(f"No matching image found for {file}")
        else:
            shutil.copy(os.path.join(source_path_labels, file), os.path.join(target_path_labels, file)) # Only add labels file if image exist.
            
# Copy files to respective directories
copy_files(train_files, images_path, labels_path, os.path.join('./datasets', 'train', 'images'), os.path.join('./datasets', 'train', 'labels'))
copy_files(val_files, images_path, labels_path, os.path.join('./datasets', 'val', 'images'), os.path.join('./datasets', 'val', 'labels'))

In [3]:
# Now you have to train using the Ultralytics
!pip install -q ultralytics

In [1]:
# https://docs.ultralytics.com/modes/train/
# Mix-up vs copy-paste: https://sh-tsang.medium.com/brief-review-simple-copy-paste-is-a-strong-data-augmentation-method-for-instance-segmentation-81297fb6f9ca
# Auto-augmentation: https://sebastianraschka.com/blog/2023/data-augmentation-pytorch.html
# external augmentation packages: https://medium.com/@k.sunman91/data-augmentation-on-ultralytics-for-training-yolov5-yolov8-97a8dab31fef
from ultralytics import YOLO
# model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')
# model = YOLO('yolov8n.yaml').load('yolov8n.pt') # build from YAML and transfer weights
results = model.train(
    data='dataset.yaml', # custom dataset
    epochs=150, # default 100 ~ higher epoch was worse went to like 38
    imgsz=640, # default 640 ~ imgsz didn't change the end result
    batch=25, # default 16,~8 was bad
    patience=9, # default 100
    dropout=0.0, # default 0.0 ~ anything on above 0.1 was bad,
    label_smoothing=0.5, # default 0.0
)

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.37 🚀 Python-3.11.6 torch-2.1.2 CUDA:0 (NVIDIA L40S, 45589MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=150, time=None, patience=9, batch=25, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

train: Scanning /projectnb/spareit/jasonoh/datasets/train/labels.cache... 10167 images, 0 backgrounds, 28 corrupt: 100%|██████████| 10167/10167 [00:00<?, ?it/s]

train: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/train/images/Compost_8279f852-089c-4988-a39d-44ddb84064c8_bb7368da-408e-43ed-91e1-95a18ff7779a_7b871bc5-ef4e-4bce-8546-a7ede2022a88.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0197      1.0814      1.0887      1.0726      1.3799      1.0626      1.0396      1.1798      1.2158       1.167      1.1614      1.3231]
train: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/train/images/Compost_a0689156-ee7e-469f-a909-42a86bd419ad_8d793a86-560c-44da-8791-733d950fea13_9688b444-7d9a-46e5-8d52-4c062d08273e.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2384      1.4041         1.1       1.006      1.3245]
train: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/train/images/Compost_af683901-0125-4b56-bb18-973465da270c_fc709174-4d39-4384-a5be-699a0613e580_efa00666-16c1-4df6-9714-65bbfc9d1dfe.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordi


val: Scanning /projectnb/spareit/jasonoh/datasets/val/labels.cache... 2010 images, 0 backgrounds, 2 corrupt: 100%|██████████| 2010/2010 [00:00<?, ?it/s]

val: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/val/images/Glass_8279f852-089c-4988-a39d-44ddb84064c8_bb7368da-408e-43ed-91e1-95a18ff7779a_f7676c9e-0a0d-42f6-b0fb-f12372fcd17b.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.4366      1.1489]
val: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/val/images/Trash_06e4cb92-5753-4a8a-bdc4-43ffad700922_07bd1196-c632-4bf4-a01f-35660b57d624_465ad689-8383-4ae0-b01f-015197a68b4b.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/val/images/Trash_9a8023cc-2608-41e3-82ed-24e7507e91d5_b7e1275d-5d0e-48b8-af8f-254c49833370_c0f2c1b2-aea6-4cab-ba9a-52b06a92963f.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /projectnb/spareit/jasonoh/datasets/val/images/Trash_a0689156-ee7e-469f-a909-42a86bd419ad_8d793a86-560c-44da-8791-733d950fea13_5793db35-fb1a-4772-ab2d-58b8b1f062a5.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.553      1.1482    

...

                   all       2008      19885      0.455      0.266      0.269      0.192

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    150/150      5.11G      1.015      1.203      1.126        156        640: 100%|██████████| 406/406 [00:34<00:00, 11.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:04<00:00,  9.38it/s]


                   all       2008      19885      0.456      0.266      0.269      0.192

150 epochs completed in 1.715 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.9MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.9MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.37 🚀 Python-3.11.6 torch-2.1.2 CUDA:0 (NVIDIA L40S, 45589MiB)
Model summary (fused): 168 layers, 3340724 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:05<00:00,  7.35it/s]


                   all       2008      19885      0.486      0.265       0.27      0.192
             Paper Cup       2008        420      0.598      0.604      0.604      0.453
Snack or Candy Bag or Wrapper       2008       1181      0.463      0.426       0.43      0.287
                  Wipe       2008          5          1          0          0          0
             Wax Paper       2008         19          0          0     0.0362     0.0298
          Latex Gloves       2008         44       0.63      0.523      0.556      0.413
  Juice or Other Pouch       2008          8          1          0    0.00833     0.0028
Padded Envelope (mixed material))       2008          3          0          0          0          0
          Blister Pack       2008          2          1          0          0          0
      Pens and Pencils       2008         16          1          0     0.0126    0.00973
Miscellaneous Office Supplies       2008          3          1          0     0.0569    0.00

In [38]:
results = model("./images/Compost_000eeeeb-0444-4f74-91e3-d6329c6e9c85_ec3a2daf-29b4-4804-9216-e6a71b07a335_35addbdb-52aa-42ac-a9c7-26ed974484f7.jpeg")


image 1/1 /projectnb/spareit/jasonoh/images/Compost_000eeeeb-0444-4f74-91e3-d6329c6e9c85_ec3a2daf-29b4-4804-9216-e6a71b07a335_35addbdb-52aa-42ac-a9c7-26ed974484f7.jpeg: 1280x960 2 Snack or Candy Bag or Wrappers, 2 Compostable Fiber Wares, 6.8ms
Speed: 8.2ms preprocess, 6.8ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 960)


In [ ]:
# Export the model to ONNX format
success = model.export(format='onnx')

In [1]:
import os
import shutil

def delete_except_sample1(directory):
    # Get all items in the directory
    items = os.listdir(directory)
    
    # Iterate through items and remove them if they are not 'sample 1'
    for item in items:
        item_path = os.path.join(directory, item)
        if item != "train7":  # Check if the current item is not 'sample 1'
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.remove(item_path)  # Remove the file or link
                print(f"Deleted file: {item_path}")
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Remove the directory
                print(f"Deleted directory: {item_path}")

# Specify the path to the directory
directory_path = './runs/detect/'
delete_except_sample1(directory_path)
print("Cleanup complete.")

Deleted directory: ./runs/detect/.ipynb_checkpoints
Deleted directory: ./runs/detect/train
Deleted directory: ./runs/detect/small-sample-train
Deleted directory: ./runs/detect/train2
Cleanup complete.
